# Calculate robusteness from pi control runs

In [33]:
# Some definitions

# directory of data
#fdir = '~/Google Drive/professional/research/FARALLON_INSTITUTE_PROJECTS/2020 NOAA MAPP/Climate_extremes_sharedfigsandcode/data/FOR SCATTER PLOT/'
fdir = '/Volumes/GoogleDrive/My Drive/Climate_extremes_sharedfigsandcode/data/Annual_TimeSeries/'

In [34]:
# Modules
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore') #filter some warning messages
import seaborn as sns

In [35]:
#detrend function: detrend use polynomial fit
def detrend(x,y,degree):
    import statsmodels.formula.api as smf
    import numpy as np
    import pandas as pd
    

    df = pd.DataFrame(columns=['y', 'x'])
    df['x'] = x
    df['y'] = pd.Series(y)   #y

    idx = np.isfinite(x) & np.isfinite(y)
    weights = np.polyfit(x[idx], y[idx], degree)
    model = np.poly1d(weights)
    results = smf.ols(formula='y ~ model(x)', data=df).fit()

    p_value=results.f_pvalue
    r2=results.rsquared_adj

    detrend_ts=np.zeros(shape=(len(y),))
    detrend_ts[:] = np.nan
    detrend_ts[idx]=y[idx]-results.fittedvalues

    #plt.figure(figsize=(10,4))
    #plt.plot(x,y)
    #plt.plot(x[idx], results.fittedvalues, 'r')
    #plt.figure(figsize=(10,4))
    #plt.plot(x, detrend_ts)
    #plt.show()

    return p_value, r2, detrend_ts

In [36]:
def num_coevents(y1, y2, pct1, pct2, runs='picontrol',loc='cclme'):
    # 
    fin1 = fdir+'original ts/'+runs+'_cclme_sst.nc'
    fin2 = fdir+'original ts/'+runs+'_goa_sst.nc'

    modsst = xr.open_dataset(fin1)
    modsst.close()
    modsm2 = xr.open_dataset(fin2)
    modsm2.close()
    modsm = xr.open_dataset(fdir+'original ts/'+runs+'_sm.nc')
    modsm.close()    
    models = modsm.model

    # selec periood
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1-1,y2-1))
    modsm3 = modsm.sel(year=slice(y1-1,y2-1))


    nev = list()
    mhw = list()
    drt = list()    
    for ix,i in enumerate(models):
        #print(ix,i)
        tmp1_0= modsst.sel(model=i).sst.values
        tmp2_0= modsm2.sel(model=i).sst.values
        tmp3_0= modsm3.sel(model=i).sm.values

        
        #detrend
        nsample = np.linspace(1,len(tmp1_0), len(tmp1_0))
        [p_value, r2, tmp1]= detrend(nsample,tmp1_0,1)
        nsample = np.linspace(1,len(tmp2_0), len(tmp2_0))
        [p_value, r2, tmp2]= detrend(nsample,tmp2_0,1)
        nsample = np.linspace(1,len(tmp3_0), len(tmp3_0))
        [p_value, r2, tmp3]= detrend(nsample,tmp3_0,1)
        

        
        # calculate threshold
        mhw_thr = np.nanpercentile(tmp1,pct1)
        drg_thr = np.nanpercentile(tmp2,pct1)
        drg_thr3 = np.nanpercentile(tmp3,pct2)   
        a1 = tmp1>=mhw_thr
        a2 = tmp2>=drg_thr
        a3 = tmp3<=drg_thr3

        
        tmp = np.full((len(modsst.year.values),1),1)
        tmp = tmp[a1*a2*a3]
        
        tmp1 = np.full((len(modsst.year.values),1),1)
        tmp1 = tmp1[a1]
        tmp2 = np.full((len(modsst.year.values),1),1)
        tmp2 = tmp2[a2]
    
        nev.append(len(tmp)/30*100) #/(y2-y1+1)) #freq.
        mhw.append(len(tmp1)/30*100)
        drt.append(len(tmp2)/30*100)        
    return nev , mhw, drt # sum(nev)

In [37]:
ny = 30
prc1 = 90
prc2 = 10
loc='cclme'

pinev=list()
pim=list()
pid=list()
for i in range(int(500/ny)):
    nev,m0,d0 = num_coevents(i*ny+2, (i+1)*ny+1, prc1 , prc2,loc=loc)
    pinev.append(nev)
    pim.append(m0)
    pid.append(d0)
pinev,np.mean(pinev) ,np.mean(pim), np.mean(pid) 

([[0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   3.3333333333333335,
   0.0,
   0.0],
  [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   3.3333333333333335,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  [0.0,
   0.0,
   3.3333333333333335,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  [3.3333333333333335,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,

In [38]:
#model by model

In [39]:
model_mean=np.mean(pinev,axis=0)
model_spread=np.std(pinev,axis=0)
IV=(np.sum((model_spread*model_spread))/22)**.5
IV,model_spread

(0.7311930716854531,
 array([1.10239638, 0.        , 1.10239638, 0.        , 0.        ,
        0.80687153, 0.80687153, 0.80687153, 0.        , 0.80687153,
        0.80687153, 0.        , 1.30104125, 0.        , 0.80687153,
        1.10239638, 1.10239638, 0.        , 0.80687153, 0.80687153,
        0.        , 0.        ]))

In [40]:
#16 30-yr periods MMEs

In [41]:
mme_mean=np.mean(pinev,axis=1)
mme_spread=np.std(pinev,axis=1)
#internal variability mean and spread
mme_mean,np.mean(mme_spread)

(array([0.15151515, 0.15151515, 0.15151515, 0.15151515, 0.        ,
        0.        , 0.15151515, 0.        , 0.15151515, 0.45454545,
        0.15151515, 0.15151515, 0.15151515, 0.15151515, 0.75757576,
        0.15151515]),
 0.6361525713738436)

In [42]:
#MME mean and spread

In [43]:
mme_model_mean=np.mean(mme_mean)
mme_model_spread=np.std(mme_mean)
mme_model_mean,mme_model_spread

(0.17992424242424243, 0.1794251450899282)

In [44]:
#cross model mean and spread

In [45]:
cross_model_mean=np.mean(model_mean)
cross_model_spread=np.std(model_mean)
cross_model_mean,cross_model_spread

(0.17992424242424243, 0.18091830657710986)

In [46]:
IV,cross_model_spread,mme_model_spread,np.std(pinev),np.std(pinev)-IV

(0.7311930716854531,
 0.18091830657710986,
 0.1794251450899282,
 0.7532428172478893,
 0.022049745562436218)

In [47]:
## Calculate thresholds for PI Control runs

In [48]:
def get_thre(loc,y1,y2,prc1,prc2):
    # open data: anomalies (without trend)

    modsst = xr.open_dataset(fdir+'original ts/picontrol_cclme_sst.nc')
    modsst.close()
    modsm2 = xr.open_dataset(fdir+'original ts/picontrol_goa_sst.nc')
    modsm2.close()
    modsm = xr.open_dataset(fdir+'original ts/picontrol_sm.nc')
    modsm.close()    
    models = modsm.model
    
    # select period 
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1-1,y2-1))
    modsm3 = modsm.sel(year=slice(y1-1,y2-1))


    mhw_thr = list()
    drg_thr = list()
    drg_thr3 = list()

    # Calculate thresholds for each model
    for ix,i in enumerate(modsm.model.values):
        tmp1_0= modsst.sel(model=i).sst.values
        tmp2_0= modsm2.sel(model=i).sst.values
        tmp3_0= modsm3.sel(model=i).sm.values

        
        #detrend
        nsample = np.linspace(1,len(tmp1_0), len(tmp1_0))
        [p_value, r2, tmp1]= detrend(nsample,tmp1_0,1)
        nsample = np.linspace(1,len(tmp2_0), len(tmp2_0))
        [p_value, r2, tmp2]= detrend(nsample,tmp2_0,1)
        nsample = np.linspace(1,len(tmp3_0), len(tmp3_0))
        [p_value, r2, tmp3]= detrend(nsample,tmp3_0,1)

        
        mhw_thr.append(np.nanpercentile(tmp1,prc1))
        drg_thr.append(np.nanpercentile(tmp2,prc1))
        drg_thr3.append(np.nanpercentile(tmp3,prc2))



    
    return mhw_thr, drg_thr ,drg_thr3 , models

In [49]:
# calculate number of events for control, historical and future
# based on pi control thresholds

def freq_coevents(fdir, runs, loc, mhw_thr, drg_thr,drg_thr3, models, y1, y2):
    #pi control
    fi1 = fdir+'original ts/'+runs+'_cclme_sst.nc'
    fi2 = fdir+'original ts/'+runs+'_goa_sst.nc'
    modsst = xr.open_dataset(fi1)
    modsst.close()
    modsm2 = xr.open_dataset(fi2)
    modsm2.close()
    modsm = xr.open_dataset(fdir+'original ts/'+runs+'_sm.nc')
    modsm.close()    
    models = modsm.model
    
    # select period
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1-1,y2-1))
    modsm3 = modsm.sel(year=slice(y1-1,y2-1))


    nev = list()
    drt=list()
    mhw=list()
    for ix,i in enumerate(models):
        #print(ix,i)
        tmp1_0= modsst.sel(model=i).sst.values
        tmp2_0= modsm2.sel(model=i).sst.values
        tmp3_0= modsm3.sel(model=i).sm.values

        
        #detrend
        nsample = np.linspace(1,len(tmp1_0), len(tmp1_0))
        [p_value, r2, tmp1]= detrend(nsample,tmp1_0,1)
        nsample = np.linspace(1,len(tmp2_0), len(tmp2_0))
        [p_value, r2, tmp2]= detrend(nsample,tmp2_0,1)
        nsample = np.linspace(1,len(tmp3_0), len(tmp3_0))
        [p_value, r2, tmp3]= detrend(nsample,tmp3_0,1)

        
        a1 = tmp1>=mhw_thr[ix]
        a2 = tmp2>=drg_thr[ix]
        a3 = tmp3<=drg_thr3[ix]
        tmp = np.full((len(modsst.year.values),1),1)
        tmp = tmp[a1*a2*a3]
        
        tmp1 = np.full((len(modsst.year.values),1),1)
        tmp1 = tmp1[a1]
        tmp2 = np.full((len(modsst.year.values),1),1)
        tmp2 = tmp2[a2]
        
        nev.append(len(tmp)/30*100)
        mhw.append(len(tmp1)/30*100)
        drt.append(len(tmp2)/30*100)
    
    freq = np.array(nev) #/(y2-y1+1)
    
    return nev,  mhw, drt #mean no. of events
#freq, np.round(np.nanmean(freq),4), np.round(np.nanstd(freq),4) #mean frequency


In [50]:
def freq_difruns(loc,ny, y0,y,prc1, prc2):
    
    # calculate threshold in pi control
    #y2=2014
    #y1=y2-ny+1
    mhw_thr, drg_thr,drg_thr3, models = get_thre(loc,y0,y,prc1,prc2)

    print ('Percentiles: '+str(prc1)+'/'+str(prc2))
    # pi control
    print('piControl: '+str(y0)+'-'+str(y))
    y2=31
    y1=2
    freq0, mfrq, sfrq = freq_coevents(fdir,'picontrol',loc, mhw_thr,drg_thr,drg_thr3, models,y0,y)
    print(mfrq,sfrq)

    # historical 1
    y1 = 1901
    y2 = y1+ny-1
    print('\nhistorical: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'historical',loc,mhw_thr,drg_thr,drg_thr3, models, y1,y2)
    print(mfrq,sfrq)


    # historical 2
    y2=2029
    y1=y2-ny+1
    print('\nhistorical 2: '+str(y1)+'-'+str(y2))
    freq1, mfrq1, sfrq1 = freq_coevents(fdir,'histssp585',loc,mhw_thr,drg_thr,drg_thr3, models, y1,y2)
    print(mfrq1,sfrq1)


    # historical 2
    y2=2014
    y1=y2-ny+1
    print('\nhistorical 2: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'historical',loc,mhw_thr,drg_thr,drg_thr3, models, y1,y2)
    print(mfrq,sfrq)

    # future 1
    y2=2099
    y1=y2-ny+1
    print('\nfuture: '+str(y1)+'-'+str(y2))
    freq2, mfrq2, sfrq2 = freq_coevents(fdir,'ssp585',loc,mhw_thr,drg_thr,drg_thr3, models, y1,y2)
    print(mfrq2,sfrq2)
    
    # future 2
    y2=2069
    y1=y2-ny+1
    print('\nfuture: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'ssp585',loc,mhw_thr,drg_thr,drg_thr3, models, y1,y2)
    print(mfrq,sfrq)

    # future 2
    y2=2045
    y1=y2-ny+1
    print('\nfuture: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'ssp585',loc,mhw_thr,drg_thr,drg_thr3, models, y1,y2)
    print(mfrq,sfrq)
    
    return freq0,freq1,freq2,mfrq1, sfrq1,mfrq2, sfrq2

In [51]:
# Frequency of events - detrended data, using pi control threshols
## percentiles 90-10

In [52]:

print ('\nPercentiles: '+str(prc1)+'/'+str(prc2))
print('\nProbable # events in '+str(ny)+' years'+': '+str(ny*0.01))
funev=list()
hinev=list()
test=list()
him=list()
hid=list()
fum=list()
fud=list()
for i in range(int(500/ny)):
    pi,hi,fu,m1,d1,m2,d2 = freq_difruns(loc,ny,i*ny+2, (i+1)*ny+1, prc1, prc2)
    test.append(pi)
    funev.append(fu)
    hinev.append(hi)
    him.append(m1)
    hid.append(d1)
    fum.append(m2)
    fud.append(d2)
np.mean(funev),np.mean(hinev),np.mean(him),np.mean(hid),np.mean(fum),np.mean(fud)


Percentiles: 90/10

Probable # events in 30 years: 0.3
Percentiles: 90/10
piControl: 2-31
[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0] [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]

historical: 1901-1930
[3.3333333333333335, 20.0, 10.0, 13.333333333333334, 3.3333333333333335, 6.666666666666667, 20.0, 6.666666666666667, 13.333333333333334, 6.666666666666667, 3.3333333333333335, 13.333333333333334, 10.0, 20.0, 23.333333333333332, 10.0, 26.666666666666668, 16.666666666666664, 10.0, 16.666666666666664, 6.666666666666667, 13.333333333333334] [13.333333333333334, 20.0, 6.666666666666667, 16.666666666666664, 10.0, 3.3333333333333335, 10.0, 10.0, 20.0, 30.0, 16.666666666666664, 6.666666666666667, 16.666666666666664, 0.0, 10.0, 10.0, 10.0, 13.333333333333334, 16.666666666666664, 10.0, 16.666666666666664, 6.666666666666667

[10.0, 6.666666666666667, 16.666666666666664, 23.333333333333332, 23.333333333333332, 10.0, 3.3333333333333335, 20.0, 6.666666666666667, 13.333333333333334, 10.0, 0.0, 3.3333333333333335, 6.666666666666667, 0.0, 20.0, 6.666666666666667, 23.333333333333332, 10.0, 10.0, 13.333333333333334, 10.0] [6.666666666666667, 3.3333333333333335, 16.666666666666664, 10.0, 16.666666666666664, 6.666666666666667, 13.333333333333334, 0.0, 26.666666666666668, 10.0, 0.0, 13.333333333333334, 16.666666666666664, 6.666666666666667, 3.3333333333333335, 13.333333333333334, 10.0, 0.0, 20.0, 0.0, 3.3333333333333335, 3.3333333333333335]
Percentiles: 90/10
piControl: 62-91
[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0] [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]

historical: 1901-1930
[6.666666666666667, 20.0, 6.666666666666667, 13.3333333333

[10.0, 6.666666666666667, 13.333333333333334, 0.0, 16.666666666666664, 13.333333333333334, 3.3333333333333335, 16.666666666666664, 6.666666666666667, 10.0, 20.0, 3.3333333333333335, 20.0, 0.0, 6.666666666666667, 13.333333333333334, 6.666666666666667, 16.666666666666664, 10.0, 20.0, 3.3333333333333335, 6.666666666666667] [10.0, 6.666666666666667, 0.0, 16.666666666666664, 13.333333333333334, 6.666666666666667, 6.666666666666667, 13.333333333333334, 23.333333333333332, 20.0, 3.3333333333333335, 3.3333333333333335, 13.333333333333334, 6.666666666666667, 10.0, 13.333333333333334, 13.333333333333334, 0.0, 10.0, 0.0, 3.3333333333333335, 13.333333333333334]
Percentiles: 90/10
piControl: 122-151
[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0] [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]

historical: 1901-1930
[6.666666666666

[16.666666666666664, 3.3333333333333335, 0.0, 0.0, 16.666666666666664, 10.0, 20.0, 20.0, 0.0, 13.333333333333334, 23.333333333333332, 10.0, 16.666666666666664, 0.0, 6.666666666666667, 6.666666666666667, 6.666666666666667, 20.0, 13.333333333333334, 3.3333333333333335, 13.333333333333334, 10.0] [6.666666666666667, 6.666666666666667, 6.666666666666667, 10.0, 16.666666666666664, 6.666666666666667, 6.666666666666667, 16.666666666666664, 13.333333333333334, 6.666666666666667, 6.666666666666667, 3.3333333333333335, 16.666666666666664, 6.666666666666667, 3.3333333333333335, 13.333333333333334, 10.0, 30.0, 23.333333333333332, 0.0, 3.3333333333333335, 10.0]
Percentiles: 90/10
piControl: 182-211
[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0] [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]

historical: 1901-1930
[10.0, 20.0, 3.33

[13.333333333333334, 10.0, 3.3333333333333335, 23.333333333333332, 23.333333333333332, 10.0, 10.0, 13.333333333333334, 10.0, 20.0, 13.333333333333334, 6.666666666666667, 6.666666666666667, 6.666666666666667, 6.666666666666667, 0.0, 13.333333333333334, 3.3333333333333335, 10.0, 23.333333333333332, 23.333333333333332, 16.666666666666664] [20.0, 0.0, 20.0, 20.0, 23.333333333333332, 23.333333333333332, 16.666666666666664, 20.0, 3.3333333333333335, 0.0, 6.666666666666667, 13.333333333333334, 10.0, 13.333333333333334, 20.0, 16.666666666666664, 13.333333333333334, 13.333333333333334, 16.666666666666664, 16.666666666666664, 6.666666666666667, 10.0]

future: 2016-2045
[16.666666666666664, 6.666666666666667, 0.0, 16.666666666666664, 20.0, 10.0, 10.0, 20.0, 0.0, 13.333333333333334, 20.0, 16.666666666666664, 16.666666666666664, 6.666666666666667, 6.666666666666667, 6.666666666666667, 10.0, 16.666666666666664, 3.3333333333333335, 20.0, 6.666666666666667, 10.0] [16.666666666666664, 0.0, 20.0, 10.0, 

[13.333333333333334, 3.3333333333333335, 13.333333333333334, 30.0, 13.333333333333334, 10.0, 0.0, 13.333333333333334, 13.333333333333334, 20.0, 13.333333333333334, 3.3333333333333335, 23.333333333333332, 6.666666666666667, 6.666666666666667, 0.0, 10.0, 10.0, 13.333333333333334, 3.3333333333333335, 23.333333333333332, 16.666666666666664] [16.666666666666664, 16.666666666666664, 16.666666666666664, 23.333333333333332, 23.333333333333332, 10.0, 16.666666666666664, 13.333333333333334, 3.3333333333333335, 20.0, 6.666666666666667, 10.0, 16.666666666666664, 16.666666666666664, 6.666666666666667, 6.666666666666667, 10.0, 10.0, 16.666666666666664, 10.0, 6.666666666666667, 3.3333333333333335]

future: 2016-2045
[10.0, 3.3333333333333335, 16.666666666666664, 23.333333333333332, 13.333333333333334, 10.0, 0.0, 16.666666666666664, 6.666666666666667, 13.333333333333334, 10.0, 3.3333333333333335, 20.0, 6.666666666666667, 6.666666666666667, 13.333333333333334, 6.666666666666667, 20.0, 10.0, 0.0, 13.333

[6.666666666666667, 6.666666666666667, 3.3333333333333335, 13.333333333333334, 20.0, 13.333333333333334, 16.666666666666664, 6.666666666666667, 3.3333333333333335, 13.333333333333334, 16.666666666666664, 16.666666666666664, 20.0, 13.333333333333334, 16.666666666666664, 10.0, 3.3333333333333335, 10.0, 6.666666666666667, 10.0, 0.0, 16.666666666666664] [13.333333333333334, 16.666666666666664, 23.333333333333332, 6.666666666666667, 10.0, 10.0, 16.666666666666664, 6.666666666666667, 6.666666666666667, 6.666666666666667, 3.3333333333333335, 6.666666666666667, 10.0, 16.666666666666664, 16.666666666666664, 13.333333333333334, 10.0, 0.0, 6.666666666666667, 16.666666666666664, 3.3333333333333335, 6.666666666666667]

future: 2040-2069
[13.333333333333334, 16.666666666666664, 3.3333333333333335, 6.666666666666667, 26.666666666666668, 20.0, 10.0, 13.333333333333334, 10.0, 20.0, 13.333333333333334, 6.666666666666667, 6.666666666666667, 10.0, 13.333333333333334, 0.0, 3.3333333333333335, 3.33333333333

[6.666666666666667, 6.666666666666667, 10.0, 20.0, 10.0, 26.666666666666668, 10.0, 6.666666666666667, 3.3333333333333335, 16.666666666666664, 10.0, 23.333333333333332, 6.666666666666667, 10.0, 16.666666666666664, 20.0, 0.0, 16.666666666666664, 13.333333333333334, 16.666666666666664, 30.0, 6.666666666666667] [10.0, 6.666666666666667, 10.0, 16.666666666666664, 3.3333333333333335, 16.666666666666664, 20.0, 10.0, 16.666666666666664, 0.0, 6.666666666666667, 13.333333333333334, 0.0, 13.333333333333334, 26.666666666666668, 13.333333333333334, 13.333333333333334, 10.0, 10.0, 16.666666666666664, 13.333333333333334, 30.0]

future: 2070-2099
[6.666666666666667, 20.0, 10.0, 13.333333333333334, 3.3333333333333335, 10.0, 16.666666666666664, 10.0, 0.0, 13.333333333333334, 6.666666666666667, 16.666666666666664, 13.333333333333334, 0.0, 16.666666666666664, 16.666666666666664, 10.0, 10.0, 10.0, 10.0, 6.666666666666667, 16.666666666666664] [0.0, 10.0, 23.333333333333332, 10.0, 6.666666666666667, 10.0, 6.

[6.666666666666667, 6.666666666666667, 10.0, 26.666666666666668, 10.0, 33.33333333333333, 10.0, 3.3333333333333335, 3.3333333333333335, 16.666666666666664, 20.0, 6.666666666666667, 20.0, 13.333333333333334, 16.666666666666664, 16.666666666666664, 0.0, 10.0, 13.333333333333334, 20.0, 26.666666666666668, 0.0] [13.333333333333334, 3.3333333333333335, 20.0, 16.666666666666664, 6.666666666666667, 3.3333333333333335, 0.0, 6.666666666666667, 16.666666666666664, 0.0, 13.333333333333334, 13.333333333333334, 13.333333333333334, 16.666666666666664, 26.666666666666668, 6.666666666666667, 23.333333333333332, 16.666666666666664, 6.666666666666667, 23.333333333333332, 6.666666666666667, 30.0]

future: 2070-2099
[6.666666666666667, 10.0, 10.0, 26.666666666666668, 10.0, 13.333333333333334, 13.333333333333334, 6.666666666666667, 3.3333333333333335, 16.666666666666664, 16.666666666666664, 3.3333333333333335, 26.666666666666668, 3.3333333333333335, 13.333333333333334, 10.0, 6.666666666666667, 10.0, 10.0, 

(0.14204545454545456,
 0.37878787878787884,
 12.481060606060606,
 14.469696969696969,
 11.145833333333332,
 11.003787878787877)

In [53]:
loc,ny

('cclme', 30)

In [54]:
N=int(500/ny)
N

16

In [55]:
np.mean(model_mean),np.mean(hinev),np.mean(funev)

(0.17992424242424243, 0.37878787878787884, 0.14204545454545456)

In [56]:
np.shape(hinev)

(16, 22)

In [57]:
#hist - picontrol FAR
diff=list()
for k in range(len(pinev)):
    zip_obj=zip(hinev[k],pinev[k])
    for i , j in zip_obj:
            diff.append(i-j)
    #diff mean, model spread
diff_a=np.asarray(diff).reshape(N,22)
diff_mean=np.mean(diff_a,axis=1)
diff_spread=np.std(diff_mean)
diff_mean.mean(),diff_spread

(0.19886363636363635, 0.2055164243769953)

In [64]:
hinev_a=np.asarray(hinev).reshape(N,22).sum(axis=1)
pinev_a=np.asarray(pinev).reshape(N,22).sum(axis=1)
far=(hinev_a-pinev_a)/hinev_a
FAR=np.mean(far)
sd=np.std(far)
FAR,sd


(0.5052083333333333, 0.5562685312144166)

In [59]:
import statsmodels.api as sm
import statsmodels as sm
import pandas
from patsy import dmatrices
cu,cl=sm.stats.proportion.proportion_confint(FAR, 1, alpha=0.05, method='normal') #'binom_test''normal'
FAR,cu,cl

(0.5052083333333333, 0.0, 1.0)

In [60]:
#future - hist change
#cross-mme spread
diff=list()
for k in range(len(pinev)):
    zip_obj=zip(funev[k],hinev[k])
    for i , j in zip_obj:
            diff.append(i-j)
    #diff mean, model spread
diff_a=np.asarray(diff).reshape(N,22)
diff_mean=np.mean(diff_a,axis=1)
diff_spread=np.std(diff_mean)
diff_mean.mean(),diff_spread

(-0.23674242424242425, 0.20021187984721567)

In [61]:
#cross-model spread
diff=list()
for k in range(len(pinev)):
    zip_obj=zip(funev[k],hinev[k])
    for i , j in zip_obj:
            diff.append(i-j)
    #diff mean, model spread
diff_mean=np.mean(diff)
diff_spread=np.std(diff)-IV
diff_mean,diff_spread

(-0.23674242424242428, 0.589108200299939)

In [62]:
funev_a=np.asarray(funev).reshape(N,22).sum(axis=1)
pinev_a=np.asarray(pinev).reshape(N,22).sum(axis=1)
far=(funev_a-pinev_a)/funev_a
FAR=np.mean(far)
sd=np.std(far)
FAR=np.mean(far)
cu,cl=sm.stats.proportion.proportion_confint(FAR, 1, alpha=0.05, method='normal') #'binom_test''normal'
FAR,cu,cl,sd

(nan, nan, nan, nan)

In [63]:
#future - pi change
#cross-mme spread
diff=list()
for k in range(len(pinev)):
    zip_obj=zip(funev[k],pinev[k])
    for i , j in zip_obj:
            diff.append(i-j)
    #diff mean, model spread
diff_a=np.asarray(diff).reshape(N,22)
diff_mean=np.mean(diff_a,axis=1)
diff_spread=np.std(diff_mean)
diff_mean.mean(),diff_spread

(-0.03787878787878788, 0.23655295448478783)